# Header for Software License

                                                         
Copyright (C) 2014-2024, Institute for Defense Analyses        
4850 Mark Center Drive, Alexandria, VA; 703-845-2500           
This material may be reproduced by or for the US Government    
pursuant to the copyright license under the clauses at DFARS   
252.227-7013 and 252.227-7014.                                 
                                                               
LARC : Linear Algebra via Recursive Compression                
Authors:                                                       
   * Steve Cuccaro (IDA-CCS)                                    
   * John Daly (LPS)                                            
   * John Gilbert (UCSB, IDA adjunct)                           
   * Jenny Zito (IDA-CCS)                                       
                                                               
Additional contributors are listed in "LARCcontributors".      
                                                               
Questions: larc@super.org                                      
                                                                
All rights reserved.                                       
                                                           
Redistribution and use in source and binary forms, with or  
without modification, are permitted provided that the
following conditions are met:                          
  * Redistribution of source code must retain the above copyright notice, this list of conditions and the following disclaimer. 
  *  Redistribution in binary form must reproduce the above  copyright notice, this list of conditions and the  following disclaimer in the documentation and/or other  materials provided with the distribution.  
  * Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission. 

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, 
INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT HOLDER NOR 
CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; 
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, 
STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR  OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

## Warning!!!

**Due to how Jupyter I/O is implemented, any printout originating from the LARC C code will be directed to the Jupyter console window (i.e., the terminal window that you typed "jupyter notebook" in), instead of in the notebook cell itself.**

## Notebook Description

This routine generates the input matrix for the power method. It first
calculates the Hamiltonian matrix H for a given potential (currently
hardcoded to the Morse ocillator), a given minimum and maximum
point on the real axis, and a discretization distance determined by the
number of points in the discretization and the interval [xmin,xmax]. 

The power method can be used to find the maximal eigvenvalue of a matrix,
but in this case we want the minimal eigenvalue. Thus we modify our
Hamiltonian matrix to invert the magnitude of the eigenvalues without
changing the eigenvectors. To do this, we assume the maximum of the
potential function on the interval is at xmin, and that therefore the
largest element in the Hamiltonian matrix is at the (0,0) location. We
use the value A of the (0,0) element as a crude estimate of the maximal
eigenvalue of the matrix. We calculate and save H'=A*I-H; the maximal
eigenvalue E'_max of H' is related to the minimal eigenvalue E_0 of H by
E_0 = A-E'_max, and the associated eigenvector is the same in both cases.

The InfoStore is used to record the value A as a string in the header of
the larcMatrix json file, where it can be read by the power method
routine. Once it finds E'_max, it can use A to calculate E_0.

In [1]:
import os
import sys 
sys.path.append("../../src")
import MyPyLARC as mypy
from ctypes import *
import math
import glob
import json    ## for loading parameter files
import re
import numpy as np

import potential
import centdiff

In [2]:
level = 10
rmax = 5.0
rmin = 0.0

verbose = 0
LARC_verbose = 0

invert_filename = "Data/invertMorse_L" + str(level) + "_rmax" + str(rmax) + ".json"
basic_filename = "Data/basicMorse_L" + str(level) + "_rmax" + str(rmax) + ".json"
evec_filename = "Data/evecMorse_L" + str(level) + "_rmax" + str(rmax)
pot_filename = "Data/morsePot_L" + str(level) + "_rmax" + str(rmax)

dim = 2**level
if (verbose > 1):
    print("\nlevel=%d: Program will use %d by %d matrices." %(level,dim,dim))
    if ((verbose == 2) or (verbose == 4)):
        print("Verbosity:")
        print("\tThe local verbosity level is verbose=%d and the" %verbose)
        print("\tverbosity level for the LARC package is %d," %LARC_verbose)
        print("\twhere 0=SILENT, 1=BASIC, 2=CHATTY, 3=DEBUG, 4=ALL.")


######################################################
##  General Description of What this Program Does   ## 
######################################################
#input("\nHit return to continue")
if (verbose > 1):
    print("\nThis code implements the power method for finding the largest")
    print("magnitude eigenvalue of the starting matrix E")
        
######################################################################
# Figure out the scalarType                                          #
# In the Makefile you can compile with different scalarType values   #
# Define string for using in formating filenames                     #
######################################################################
scalarTypeStr = mypy.cvar.scalarTypeStr

######################################################
##   Find out if machine has a large amount of      ##
##   memory available so we can make bigger tables  ##
######################################################
memory_available = mypy.memory_available_GiB()
if (verbose > 0):
    print("\nThe memory available is %ld GiB" %memory_available)
    print("We will use this to select which computing_env to read from parameter file.")
    print("You could write code to select computing_env automatically.")

if (memory_available > 200):
    if (verbose > 0):
        print("\nThis memory is more than 200 GiB\n")
    computing_env = 'large'
else:    
    if (memory_available > 50):
        if (verbose > 0):
            print("\nThis memory is between 50 and 200 GiB\n")
        computing_env = 'medium'
    else:
        if (verbose>0):
            print("\nThis memory is less than 50 GiB\n")
        computing_env = 'small'
        if (memory_available < 10):
            print("\nYou have less than 10 GiB of memory available: defaulting to")
            print("small toy case to avoid overloading your computer\n")
            level = 3
            
if (verbose > 0):
    print("This program believes the computing_environment is %s" %computing_env)


#######################################
##    Print baseline usage report    ##
#######################################
#userInput= input("Press <Enter> to continue\n")
if (verbose > 0):
    print("")
    print("In the following baseline usage report")
    print("RSS, resident set size, refers to size of the process held in RAM.")
    print("HASHSTATS: hash occupancy means, variances and crash resolution chain lengths")
    mypy.memory_and_time_report(0, "stdout")
    print("   SEE JUPYTER CONSOLE FOR USAGE REPORT!")

## read the parameter file into a python dictionary
with open('../../InitParams/power_method.init_params','r') as init_file:
    init_param = json.load(init_file)
    for p in init_param[computing_env]:
        if (verbose > 1):
            print('MatrixExponent: %d' %(p['matrix_exponent']))
            print('OpExponent: %d' %(p['op_exponent']))
            print('MaxLevel: %d' %(p['max_level']))
            print('RegionBitParam: %d' %(p['regionbitparam']))
            print('ZeroRegionBitParam: %d' %(p['zeroregionbitparam']))
            print('ReportIntervalSecs: %d' %(p['report_interval_seconds']))
            print('MinMemRequiredGiB: %d' %(p['min_memGiB_required']))
            print('Verbose: %d' %(p['verbose']))
            print('')
        matrix_exponent = p['matrix_exponent']
        op_exponent = p['op_exponent']
        max_level= p['max_level']
        regionbitparam = p['regionbitparam']
        zeroregionbitparam = p['zeroregionbitparam']
        report_interval_seconds = p['report_interval_seconds']
        min_memGiB_required = p['min_memGiB_required']
        p_verbose = p['verbose']

## warn if the commandline value for verbose differs from the parameter file value for verbose        
if (verbose > 0):
    if (verbose != p_verbose):
        print("NOTE: This program uses commandline (verbose = %d) " %verbose)
        print("      rather than the parameter file (verbose = %d)." %p_verbose)
        print("      The verbose key is:  0=SILENT, 1=BASIC, 2=CHATTY, 3=DEBUG.")

## initialize LARC
mypy.initialize_larc(matrix_exponent,op_exponent,max_level,regionbitparam,zeroregionbitparam,LARC_verbose)


You have less than 10 GiB of memory available: defaulting to
small toy case to avoid overloading your computer



In [3]:
# We are going to discretize the real line between the two points rmin
# and rmax. There will be a total of 2**level points in the discretization,
# evenly spaced; the distance between adjacent points is given by delta.
npts = 2**level
delta = (rmax-rmin)/(npts-1)

# create a list of r values to pass into the potential function
r = [rmin + i*delta for i in range(npts)]

# the morse oscillator hamiltonian is
# given by -\hbar^2/2m d^2/dr^2 + V_{morse}(r;D_e,r_e,\alpha)
# where: D_e is the well depth of the potential
#           (thus -D_e is the minimum energy of the potential)
#        r_e is the distance at which this minimum is achieved
#           (the equilibrium bond distance)
#        \alpha controls the width of the well

# create and store the potential matrix
# one set of suitable parameters
#            r_e = 1.28 Å, D = 4.17 eV and α = 1.85 Å^{-1}
# (Anil K. Shukla, Jean H. Futrell, "Ion Collision Theory", in
# Encyclopedia of Spectroscopy and Spectrometry, 1999, Academic Press,
# pp. 954--963:
# "the Morse potential with these parameters matches the experimentally
# determined scattering of protons by argon to better than 1% accuracy over
# the interval from 0 to 5 Å.")

# note that the reference listed the α units as Å^3, but this is 
# obviously incorrect since it appears in the exponential multiplied by
# a distance, not an inverse distance cubed.

# now for the fun part: getting everything into consistent units
# we will use Angstroms (1Å = 10^{-10} meters) as our distance unit,
# and electron-volts (1eV = 1.60218e-19 Joules) as our energy unit.

r_min_energy = 1.28
well_depth = 4.17
alpha = 1.85
vr = potential.morse_potential(r,alpha,well_depth,r_min_energy)
vr_str = mypy.map_to_str(vr, scalarTypeStr)
potVecID = mypy.row_major_list_to_store(vr_str, level, 0, 1)
# potVecID is a column vector, but we need to put the potential energy
# values on the diagonal of a 2^level x 2^level matrix
potMatID = potential.put_col_vector_on_diagonal(potVecID,level)
#print("potMatID:")
#mypy.print_naive(potMatID)

# save the potential to disk for future plotting
file1 = open(pot_filename,'w')
for vpt in vr:
    file1.write(str(vpt)+"\n")
file1.close()

In [4]:
# create and store the kinetic (second derivative) matrix
# from Wikipedia
# speed of light c = 2.99792458e18 Å/s 
# hbar = 6.582119569e−16 eV⋅s
# mass of a proton: 1.007276466621(53) amu = 9.3827208816(29)e8 eV/c^2
# mass of Argon: 39.9623831238(24) amu
# reduced mass of two particles is given by (1/m1 + 1/m2)^{-1}:
# (1/1.007276466621 + 1/39.9623831238)^{-1)
#      = ( 0.992776098 + 0.025023533 )^{-1}
#      = ( 1.017799631 )^{-1} =  0.982511656 amu

# => conversion factor, calculated from proton mass numbers:
#    931494102.416110814 (eV/c^2)/amu
# using conversion factor, reduced mass is 9.15203813e8 eV/c^2

# units of -hbar^2/2m: (eV*s)^2/(eV/c^2) == eV*s^2/c^2
# -hbar^2/2m = -(6.582119569e-16)^2/2.0/9.15203813e8 
#            = -2.366920756e-40 eV⋅s^2/c^2
#  * c^2     => -2.1272822872e-3 eV⋅Å^2

ddr2ID = centdiff.build_central_diff_matrix(level,delta)
scale_factor = -0.0021272822872
scaleStr = mypy.value_to_string(scale_factor,scalarTypeStr)
scaleID = mypy.get_valID_from_valString(scaleStr)
kinMatID = mypy.scalar_mult(scaleID,ddr2ID)
#print("kinMatID:")
#mypy.print_naive(kinMatID)

# add to get Hamiltonian matrix
hamMatID = mypy.matrix_add(kinMatID,potMatID)
#print("hamMatID:")
#mypy.print_naive(hamMatID)
mypy.fprint_larcMatrixFile(hamMatID, basic_filename)

# modify as describe above so that smallest eigenvalue becomes largest
extremeStr = mypy.get_readableString_scalar_from_pID_and_coords(hamMatID,0,0)
print("extreme value is ", extremeStr)
extremeID = mypy.get_scalarID_from_pID_and_coords(hamMatID,0,0)
#mypy.print_naive(extremeID)
bigDiagID = mypy.scalar_mult(extremeID,
                 mypy.get_identity_pID(level))
invHamID = mypy.matrix_diff(bigDiagID,hamMatID)
#print("invHamID:")
#mypy.print_naive(invHamID)

#write the value of extremeID into the infoStore so that the 
#power method will have access to it
mypy.info_set(mypy.OTHERINFO,invHamID,
    "subtract the power-method-calculated eigenvalue from this to" +
    " get the correct eigenvalue for the ground state of the Hamiltonian")
mypy.info_set(mypy.OTHERMATRIX,invHamID,
     mypy.get_scalar_value_string(extremeID))

#help the power method by giving it a good starting point: tell it to use
#the unit vector with a "1" in a position near the minimum energy
minpos = int((r_min_energy-rmin)/delta)
mypy.info_set(mypy.COMMENT,invHamID, str(minpos) +
        " is a good position for the nonzero in the starting vector")

# write the matrix to disk
dummy = mypy.fprint_larcMatrixFile(invHamID, invert_filename)

extreme value is  386.25601244902852727+I*0


In [5]:
if scalarTypeStr in ('Integer', 'MPInteger'):
    print("\nThis routine does not work with integer types!")
    sys.exit(1)
if scalarTypeStr in ('MPRational','MPRatComplex'):
    print("\nThis routine does not give good results with this scalarType")
    print("Consider using either MPReal or MPComplex instead")

## if verbosity higher than WARN start a reporting thread              
if (verbose > 1):              
    mypy.create_report_thread(report_interval_seconds)

#########################################
## read in inverted Hamiltonian matrix ##
#########################################

invHamID = mypy.read_larcMatrixFile(invert_filename)
adjustEigStr = mypy.info_get(mypy.OTHERMATRIX,invHamID)
print("the adjustment value is %s" %adjustEigStr)
adjustEigID = mypy.get_valID_from_valString(adjustEigStr)
startposStr = mypy.info_get(mypy.COMMENT,invHamID)
print(startposStr)
startposList = startposStr.split(" ")
startpos = int(startposList[0])

############################################################
## if matrices are too large do not allow naive printing  ##            
############################################################
if (level < 4):
    print_naive = 1
else:
    print_naive = 0
if (verbose > 1):             
    if print_naive:
        print("  The level is small enough that we can print files of naive matrices to the screen.")
    else: 
        print("  The level= %d, is too big to reasonable print naive formated matrices to the screen." %level)

print("\n We have loaded the matrix into LARC and it has matrixID %d\n"
         %invHamID)
print("\n")

# We will make a simple vector to start
# use the input from the infoStore to choose a good unit vector
v = [0 for i in range (0,dim)]
v[startpos] = 1
#print("\n We have created the vector v")
#print(v)
        
#a_arr = list(map(str,a_str))
v_str_list = mypy.map_to_str(v,scalarTypeStr)
#print("\n We tried to turn v into a list of strings")
#print(v_str_list)

# creating or finding the matrix associated with the array
v_ID = mypy.row_major_list_to_store(v_str_list, level, 0, 1)
# mypy.print_naive(v_ID)
print("\n We have loaded the column vector v into LARC and it has matrixID %d\n"%v_ID)
print("\n")

v_old_ID = -1
v_new_ID = v_ID
# the maximum element in the unit vector is "1", which is pre-stored
one_ID = mypy.get_identity_pID(0)
maxEl_ID = one_ID
counter = 0


# TODO: Add query for which norm you want to use
maxnorm = 1  # 1 means use the maxnorm, versus L2 norm
if (maxnorm):
    chosen_norm = mypy.L_infty
else:
    chosen_norm = mypy.l_2


the adjustment value is 386.25601244902852727+I*0
1 is a good position for the nonzero in the starting vector

 We have loaded the matrix into LARC and it has matrixID 251




 We have loaded the column vector v into LARC and it has matrixID 254





In [6]:
# retrieve the next matrix ID to be assigned in LARC v_first_ID
v_first_ID = mypy.num_matrices_created()
# make exit condition that v_first_ID < v_new_ID < v_old_ID

# MARK claims: v_norm decreasing is also a stopping condition

loop_hash_table = set()

print("\nStarting power method loop!")
#while (v_old_ID != v_new_ID): # (v_old_ID = v_new_ID) converged  
while (1):
    v_old_ID = v_new_ID
    old_maxEl_ID = maxEl_ID
    v_unnorm_ID = mypy.matrix_mult(invHamID,v_old_ID)
    maxEl_ID = mypy.matrix_element_with_maxNorm(v_unnorm_ID)
    # eventually, the eigenvalue will be given by the ratio
    # of the new maximum ID element and the old maximum ID element
    eval_ID = mypy.scalar_divide(maxEl_ID,old_maxEl_ID)
    if (maxnorm):
        # dividing by the element with maximal norm makes the
        # value of this element in the vector equal to 1.0
        # (+ 0.0i if complex); it remains maximal
        v_new_ID = mypy.scalar_divide(v_unnorm_ID, maxEl_ID)
        # when using l\infty norm, the vector is now normalized
        maxEl_ID = one_ID
    else:
        # dividing by the element with maximal norm makes the
        # value of this element in the vector equal to 1.0
        # (+ 0.0i if complex); it remains maximal
        v_temp_ID = mypy.scalar_divide(v_unnorm_ID, maxEl_ID)
        # when using l2 norm, we still need to normalize
        norm_ID = mypy.normID(v_temp_ID,mypy.L_2)
        v_new_ID = mypy.scalar_divide(v_temp_ID,norm_ID)
        maxEl_ID = mypy.scalar_divide(one_ID,norm_ID)

    # print("The normalized vector has ID %d\n" %normalizedID1)
    # mypy.print_naive(normalizedID1)
    if (v_new_ID == v_old_ID):
        evalue_str = mypy.get_scalar_value_string(eval_ID)
        print("Exiting: new and old eigenvectors are the same!")
        break
    if (v_new_ID < v_old_ID):
        if (v_new_ID in loop_hash_table):
            evalue_str = mypy.get_scalar_value_string(eval_ID)
            print("Exiting: detected loop convergence.")
            break
        else:
            print("\n\tnew is %d, old is %d" %(v_new_ID,v_old_ID))
            print("\tadding ID %d to hash table" %v_new_ID)
            loop_hash_table.add(v_new_ID)
    counter += 1
    if (counter%1000 == 0):
        evalue_str = mypy.get_scalar_value_string(eval_ID)
        #print("The counter is %d" %counter)
        print("The counter is %d, evalue_str %s"
            %(counter, evalue_str))
        # print("The IDs for v_old is %d and it is" %v_old_ID)
        # mypy.print_naive(v_old_ID)
        # print("The ID for v_new is %d and it is" %v_new_ID)
        # mypy.print_naive(v_new_ID)
        print("The ID for v_new is %d" %v_new_ID)

print("After loop the counter is %d, the eigenvalue is %s" %(counter,evalue_str))
#print("and the eigenvector is")
#mypy.print_naive(v_new_ID)

# evString = "eigenvector_L" + str(level)
mypy.fprint_naive(v_new_ID,evec_filename)

v_diff_ID = mypy.matrix_diff(v_new_ID,v_old_ID)
#v_diff_norm_str = chosen_norm(v_diff_ID)
v_diff_norm_ID = mypy.normID(v_diff_ID,chosen_norm)
v_diff_norm_str = mypy.traceID(v_diff_norm_ID)
print("The difference v_new and v_old has norm %s" %v_diff_norm_str)
#mypy.print_naive(v_diff_ID)
    
# subtract the eigenvalue from adjustEig to get the correct eigenvalue
correctEigID = mypy.matrix_diff(adjustEigID,eval_ID)
print("the adjusted eigenvalue is ",
      mypy.get_scalar_value_string(correctEigID))


Starting power method loop!
The counter is 1000, evalue_str 390.17685028529888919+I*0
The ID for v_new is 55058
The counter is 2000, evalue_str 390.17685030311310607+I*0
The ID for v_new is 104329
The counter is 3000, evalue_str 390.17685030314937847+I*0
The ID for v_new is 139277
The counter is 4000, evalue_str 390.17685030314945233+I*0
The ID for v_new is 162757
Exiting: new and old eigenvectors are the same!
After loop the counter is 4453, the eigenvalue is 390.17685030314945246+I*0
The difference v_new and v_old has norm 0+I*0
the adjusted eigenvalue is  -3.9208378541209251977+I*0
